## Cài đặt BigchainDB

In [ ]:
!pip3 install --upgrade setuptools
!sudo apt-get update
!sudo apt-get install python3-dev libssl-dev libffi-dev
!pip3 install bigchaindb_driver

     |████████████████████████████████| 952 kB 5.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://developer.download.nvidia.com/compute/c

## Import thư viện và Kết nối BigchainDB

In [ ]:
from bigchaindb_driver import BigchainDB
from bigchaindb_driver.common.utils import serialize
from bigchaindb_driver.crypto import generate_keypair
import json
 
bdb_root_url = 'https://test.ipdb.io'
 
bdb = BigchainDB(bdb_root_url)

user = generate_keypair()

print(user)

CryptoKeypair(private_key='7PWsNGPGaQMKiozY8cobbi4KhooUzp8hs4d4vR2vqKng', public_key='6G2YHpG9CGzxJdrWn8cAsr1V3F7fW5c1wYs7BfEW8nV9')


## Kết nối google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Đọc dữ liệu

Thay data_file bằng đường link dẫn đến dữ liệu

In [ ]:
import json

data_file = "/content/drive/Shareddrives/SE-ex/rv_listing_dataset.json"
data = json.load(open(data_file, "r", encoding="utf8"))
data=data[1008:1020]
print(json.dumps(data, indent=4, ensure_ascii=False))

[
    {
        "balcony_direction": 8,
        "furniture_status": "Empty",
        "has_video": false,
        "num_bed_room": 2,
        "area": 79.1,
        "name": "Tầng 12, Block B, D'Lusso",
        "created_time": 1606537987981,
        "full_name": "B12.10 Tầng 12, Block B, D'Lusso",
        "property_type": 1,
        "tags": [
            "căn hộ D'Lusso",
            "căn hộ quận 2",
            "căn hộ 2 phòng ngủ"
        ],
        "direction": 6,
        "published_time": 1606545733625,
        "alias": "can-ho-d-lusso-block-b-tang-12-view-thoang-mat-khong-noi-that",
        "juridical_status": 0,
        "house_status": 0,
        "amenities": {
            "basement": false,
            "smart_home": false,
            "tv": false,
            "fridge": false,
            "relax_room": false,
            "store_house": false,
            "smart_drying_rig": false,
            "gas_stove": false,
            "mini_bar": false,
            "cable": false,
            "

## Functions

In [ ]:
##  dict_values:    1 dòng trong file json
def add(dict_values):
    addAsset = {}
    addAsset['data']=dict_values
    # pass

    prepare_creation = bdb.transactions.prepare(
        operation='CREATE',
        signers=user.public_key,
        asset=addAsset,
    )
    # print('PREPARE:')
    # print(json.dumps(prepare_creation, indent=4))
    # print()

    fulfilled_creation = bdb.transactions.fulfill(
        prepare_creation, user.private_key
    )
    
    # print('FULFILL:')
    # print(json.dumps(fulfilled_creation, indent=4))
    # print()
        
    # Send the transaction to BigchainDB
    sent_creation = bdb.transactions.send_commit(fulfilled_creation)
    return fulfilled_creation["id"]

##  Change asset fields given in fields variable, field maybe is {field_name:value} or {field_name:dict}
def set_asset_fields(asset, fields):
  if fields == {}:
    return
  for field in fields.keys():
    value = fields[field]
    if type(value) is dict:
      set_asset_fields(asset[field], value)
    else:
      asset[field] = value

def get_asset(asset_id):
  try:
    transactions = bdb.transactions.get(asset_id=asset_id)
  except:
    print("Unable to get transactions!")
    return
  if transactions == [] or transactions is None:
    return {}
  asset = {}
  for transaction in transactions:
    if asset == {}:
      asset = transaction["asset"]["data"]
    else:
      set_asset_fields(asset, transaction["metadata"]["data"])
  return asset

def update(asset_id, input_data_changed):
    transfer_asset = {
        'id' : asset_id
    }
    data_changed = {}
    data_changed['data']=input_data_changed

    # data_changed = {
    #     'data' : {
    #         'Owner' : 111222333
    #     }
    # }
    
    output_index = 0
    
    transactions = bdb.transactions.get(asset_id=asset_id)
    last_transaction = transactions[-1]

    # output = fulfilled_creation['outputs'][output_index]
    output = last_transaction['outputs'][output_index]
    
    transfer_input = {
        'fulfillment': output['condition']['details'],
        'fulfills': {
        'output_index': output_index,
        'transaction_id': last_transaction['id'],
        },
        'owners_before': output['public_keys'],
    }
    
    prepare_creation = bdb.transactions.prepare(
        operation='TRANSFER',
        asset = transfer_asset,
        inputs = transfer_input,
        metadata= data_changed,
        recipients=user.public_key
    )
    
    last_transaction = bdb.transactions.fulfill(
        prepare_creation,
        private_keys=user.private_key
    )
    
    sent_creation = bdb.transactions.send_commit(last_transaction)
    return last_transaction["id"]
  

## Examples

In [ ]:
idx = add(data[0])
get_asset(idx)

{'address': {'city': 'Hồ Chí Minh',
  'city_id': '1576833711767_2408',
  'district': 'Quận 2',
  'district_id': '1576833722050_9077',
  'floor_number': '12',
  'geolocation': {'lat': 10.789582, 'lon': 106.75485600000002},
  'neighborhood_id': '',
  'no_serialize': '',
  'street': 'Nguyễn Thị Định',
  'street_id': 'nguyen-thi-dinh',
  'tower_id': '8ededa3a-709b-4a39-a953-c6dec8934b78',
  'unit_number_serialize': 'b1210',
  'ward': 'Phường An Phú',
  'ward_id': '1578033928606_854'},
 'alias': 'can-ho-d-lusso-block-b-tang-12-view-thoang-mat-khong-noi-that',
 'amenities': {'air_conditioner': False,
  'basement': False,
  'cable': False,
  'dryer': False,
  'fingerprint_lock': False,
  'fire_detection': False,
  'fridge': False,
  'garage': False,
  'garden': False,
  'gas_stove': False,
  'gym': False,
  'helper_room': False,
  'internet': False,
  'kitchen_hood': False,
  'magnetic_card_lock': False,
  'microwave': False,
  'mini_bar': False,
  'oven': False,
  'password_lock': False,
  '

In [ ]:
data_tochange = {
    'amenities' : {
        'garage' : True,
        'garden' : True
    },
    'has_video' : True
}
update(idx, data_tochange)
get_asset(idx)

{'address': {'city': 'Hồ Chí Minh',
  'city_id': '1576833711767_2408',
  'district': 'Quận 2',
  'district_id': '1576833722050_9077',
  'floor_number': '12',
  'geolocation': {'lat': 10.789582, 'lon': 106.75485600000002},
  'neighborhood_id': '',
  'no_serialize': '',
  'street': 'Nguyễn Thị Định',
  'street_id': 'nguyen-thi-dinh',
  'tower_id': '8ededa3a-709b-4a39-a953-c6dec8934b78',
  'unit_number_serialize': 'b1210',
  'ward': 'Phường An Phú',
  'ward_id': '1578033928606_854'},
 'alias': 'can-ho-d-lusso-block-b-tang-12-view-thoang-mat-khong-noi-that',
 'amenities': {'air_conditioner': False,
  'basement': False,
  'cable': False,
  'dryer': False,
  'fingerprint_lock': False,
  'fire_detection': False,
  'fridge': False,
  'garage': True,
  'garden': True,
  'gas_stove': False,
  'gym': False,
  'helper_room': False,
  'internet': False,
  'kitchen_hood': False,
  'magnetic_card_lock': False,
  'microwave': False,
  'mini_bar': False,
  'oven': False,
  'password_lock': False,
  'pe

## Example Usage

In [ ]:
data_searched = bdb.assets.get(search="D'Lusso")

if data_searched == []:
  print("not found")
else:
  for item in data_searched:
    print(json.dumps({"data" : get_asset(item["id"]), "id" : item["id"]}, ensure_ascii=False, indent=4))

{
    "data": {
        "balcony_direction": 8,
        "furniture_status": "Empty",
        "has_video": true,
        "num_bed_room": 2,
        "area": 79.1,
        "name": "Tầng 12, Block B, D'Lusso",
        "created_time": 1606537987981,
        "full_name": "B12.10 Tầng 12, Block B, D'Lusso",
        "property_type": 1,
        "tags": [
            "căn hộ D'Lusso",
            "căn hộ quận 2",
            "căn hộ 2 phòng ngủ"
        ],
        "direction": 6,
        "published_time": 1606545733625,
        "alias": "can-ho-d-lusso-block-b-tang-12-view-thoang-mat-khong-noi-that",
        "juridical_status": 0,
        "house_status": 0,
        "amenities": {
            "basement": false,
            "smart_home": false,
            "tv": false,
            "fridge": false,
            "relax_room": false,
            "store_house": false,
            "smart_drying_rig": false,
            "gas_stove": false,
            "mini_bar": false,
            "cable": false,
      

### Choose an asset

In [ ]:
asset_chosen_id = data_searched[-1]["id"]
print(asset_chosen_id)

2d6e6e4ae5d671dece6c249c879b11bfd224ff5f63d5fd786ae2dc654e47f6e9


### Update the chosen asset

In [ ]:
update(asset_chosen_id, {"furniture_status" : "Full"})
print(json.dumps(get_asset(asset_chosen_id), indent=4, ensure_ascii=False))

{
    "balcony_direction": 8,
    "furniture_status": "Full",
    "has_video": true,
    "num_bed_room": 2,
    "area": 79.1,
    "name": "Tầng 12, Block B, D'Lusso",
    "created_time": 1606537987981,
    "full_name": "B12.10 Tầng 12, Block B, D'Lusso",
    "property_type": 1,
    "tags": [
        "căn hộ D'Lusso",
        "căn hộ quận 2",
        "căn hộ 2 phòng ngủ"
    ],
    "direction": 6,
    "published_time": 1606545733625,
    "alias": "can-ho-d-lusso-block-b-tang-12-view-thoang-mat-khong-noi-that",
    "juridical_status": 0,
    "house_status": 0,
    "amenities": {
        "basement": false,
        "smart_home": false,
        "tv": false,
        "fridge": false,
        "relax_room": false,
        "store_house": false,
        "smart_drying_rig": false,
        "gas_stove": false,
        "mini_bar": false,
        "cable": false,
        "pet": false,
        "microwave": false,
        "helper_room": false,
        "washing_machine": false,
        "oven": false,
    